In [68]:
import pandas as pd
class2 = pd.read_csv('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/class2.csv')

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

In [69]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [70]:
glove_file = datapath('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/glove.6B.100d.txt')

In [71]:
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file,word2vec_glove_file)

C:\Users\516-29\AppData\Local\Temp\ipykernel_15736\3214318432.py:2: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file,word2vec_glove_file)


(400000, 100)

In [73]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

model.most_similar('bill')

[('legislation', 0.8072139620780945),
 ('proposal', 0.730686366558075),
 ('senate', 0.7142541408538818),
 ('bills', 0.704440176486969),
 ('measure', 0.6958035230636597),
 ('passed', 0.690624475479126),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663140058517456)]

In [74]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


In [104]:
from __future__ import unicode_literals, print_function, division 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [105]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addword(word)
    def addword(self,word):
        if word not in self.word2index:
            self.word2index[word] =self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] +=1

In [106]:
def normalizeString(df,lang):
    sentence  =df[lang].str.lower()
    sentence = sentence.str.replace('[^A-Za-z\s]+',' ')

    sentence = sentence.str.normaliz('NFD')
    sentence = sentence.str.encode('ascil', error = 'ignore').str.decode('uft-8')
    return sentence
def read_sentence(df,lang1,lang2):
    sentence1 = normalizeString(df,lang1)
    sentence2 = normalizeString(df,lang2)
    return sentence1, sentence2
def read_file(loc,lang1,lang2):
    df = pd.read_csv('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/%s-%s.txt' % (lang1,lang2), lang1,lang2)
    sentence1,sentence2 = read_sentence(df,lang1,lang2)
    input_lang = Lang()
    output_lang = Lang()
    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]]
            input_lang.addSentence(sentence1[i])
            output_lang.addSentence(sentence2[i])
            pairs.append(full)
            
                    
    return input_lang,output_lang,pairs                     

In [107]:
def indexFromSentence(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang,sentence):
    indexes = indexFromSentence(lang,sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(-1,1)

def tensorFromSentence(input_lang, output_lang,pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    output_tensor = tensorFromSentence(output_lang, pair[1])
    return(input_tensor, output_tensor)

In [108]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
         super(Encoder, self).__init__()
         self.input_dim = input_dim
         self.embbed_dim = embbed_dim
         self.hidden_dim = hidden_dim
         self.num_layers = num_layers
         self.embedding = nn.Embedding(input_dim,self.embbed_dim)
         self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers = self.num_layers)
    def forward(self, src):
        embedded = self.embbdding(src).view(1,1,-1)
        outputs,hidden = self.gru(embedded)
        return outputs,hidden

In [109]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder,self).__init__()

        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers = self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input,hidden):
        input = input.view(1,-1)
        embedded = F.relu(self.embedding(nput))
        outputs,hidden = self.gru(embedded,hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden    
        

In [110]:
class Seq25eq(nn.Module):
    def __init__(self,encoder, decoder, device, MAX_LENGTH = MAX_LENGTH):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,input_lang,output_lang, teacher_forcing_ratio = 0.5):
        input_length = input_lang.size(0)
        batch_size = output_lang.shape[1]
        target_length = output_lang.shape[0]
        vocab_size = self.decoder.output_dim
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

        for i in range(input_length):
            encoder_output, encoder_hidden = self.encoder(input_lang[i])
            decoder_hidden = encoder_hieedn.to(device)
            decoder_input = torch.tensor( [SOS_token], device = device)
        for t in range(target_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            input = (output_lang[t] if teacher_force else topi)
            if(teacher_force == False and input.item() == EOS_token):
                break
        return outputs

In [111]:
teacher_forcing_ratio = 0.5

def Model(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0
    output = model(input_tensor, target_tensor)
    num_iter = output_size(0)

    for ot in range(num_iter):
        loss += criterion(output[ot],  target_tensor[ot])
    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter
    return epoch_loss

In [112]:
def trainModel(model,input_lang, output_lang, pairs, num_iteration=20000):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr = 0.01)
    criterion = nn.NLLLoss()
    total_loss_iterations = 0

    training_pairs = [tensorFromSentence(input_lang, output_lang, random.choice(pairs)) for i in range(num_iteration)] 

    for iter in range(1,num_iteration + 1):
        training_pair = training_pars[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = Model(model, input_tensor, target_tensor, optimizer, criterion)
        total_loss_iteration += loss

        if iter % 5000 == 0:
            average_loss = total_loss_iterations / 5000
            total_loss_iterations = 0
            print('%d %.4f' % (iter,average_loss))
    torch.save(model.state_dict(), '/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/mytraining.pt')
    return model

In [113]:
def evaluate(model, input_lang, output_lang, sentences, max_length = MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])
        decoded_words = []
        output = model(input_tensor, output_tensor)

        for ot in range(output.size(0)):
            topv,topi = output[ot].topk(1)
            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words

def evaluateRandomly(model,input_lang,output_lang, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('input{}'.format(pair[0]))
        print('output{}'.format(pair[1]))
        output_words = evaluate(model,input_lang, output_lang, pair)
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))

In [114]:
lang1 = 'eng'
lang2 = 'fra'
input_lang, output_lang,pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

input_size = input_lang.n_words
output_size = output_lang.n_words
embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 75000

encoder = Encoder(input_size,hidden_size,embed_size, num_layers)
decoder = Decoder(output_size,hidden_size,embed_size, num_layers)
model = Seq25eq(encoder, decoder, device).to(device)

print(encoder)
print(decoder)

model = trainModel(model,input_lang,output_lang,pairs,num_iteration)


NameError: name 'process_data' is not defined

In [115]:
class AttenDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,dropout_p=0.1, max_length = MAX_LENGTH):
        super(AttenDecoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
    def forward(self,input,hidden,ecoder_outputs):
        embedded = self.embedding(input).view(1,1,-1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat( (embedded[0], hieed[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat( (embedded[0],attn_applied[0]),1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

In [100]:
def trainIters(encoder, decoder, n_iters, print_every = 1000, plot_every = 100, learning_rate = 0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    training_pairs = [tensorFromSentence(input_lang, output_lang, random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1,n_iters + 1):
        training_pair = training_pairs[iter-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = Model(model, input_tensor, target_tensor, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % 5000 == 0:
            print_loss_avg = print_loss_total / 5000
            print_loss_total = 0
            print('%d, %.4f' % (iter, print_loss_avg))

In [116]:
import time
embed_size = 128
hidden_size = 256
num_layers = 1
input_size = input_lang.n_words
output_size = output_lang.n_words

encoder1 = Encoder(input_size,hidden_size, embed_size, num_layers)
attn_decoder1 = AttenDecoderRNN(hidden_size, output_size, dropout_p = 0.1).to(device)
print(encoder1)
print(attn_decoder1)
attn_model = trainIters(encoder1,attn_decoder1,75000, print_every = 5000, plot_every = 100, learning_rate = 0.01)

NameError: name 'input_lang' is not defined

In [120]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification 
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [122]:
train_df = pd.read_csv('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/training.txt', sep = '\t')
valid_df = pd.read_csv('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/validing.txt', sep='\t')
test_df = pd.read_csv('/Users/516-29/dip/딥러닝_파이토치_교과서_예제파일_(2)/chap10/data/testing.txt',sep = '\t')

In [123]:
train_df = train_df.sample(frac = 0.1, random_state = 300)
valid_df = valid_df.sample(frac = 0.1, random_state = 300)
test_df = test_df.sample(frac = 0.1, random_state = 300)


In [124]:
class Datasets(Dataset):
    def __init__(self,df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.iloc[idx,1]
        label = self.df.iloc[idx,2]
        return text,label

In [125]:
train_dataset = Datasets(train_df)
train_loader = DataLoader(train_dataset,batch_size = 2, shuffle = True, num_workers = 0)

valid_dataset = Datasets(valid_df)
valid_loader = DataLoader(valid_dataset,batch_size=2, shuffle = True, num_workers=0)

test_dataset = Datasets(test_df)
test_loader =  DataLoader(test_dataset,batch_size=2, shuffle = True, num_workers=0)

In [127]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

100%|████████████████████████████████████████████████████████████████| 440473133/440473133 [02:11<00:00, 3345210.62B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [130]:
def save_checkpoint(save_path,model,valid_loss):
    if save_path == None:
        return
    state_dict = {'model_state_dict':model.state_dict(), 'valid_loss':valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    if load_path == None:
        return
    state_dict = torch.load(load_path,map_location=device)
    print(f'Model loadded from <== {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):
    if save_path == None:
        return
    state_dict = {'train_loss_lost': train_loss_list,
                 'valid_loss_list': valid_loss_list,
                 'global_steps_list': global_steps_list}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save+path}')

def load_metrics(load_path):
    if load_path == None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']